In [1]:
#########################################################################################################################
# PROGRAMA WEB SCRAPING - SISTEMA DE SEGUIMIENTO DE INVERSIONES
#########################################################################################################################

# Instalar paquetes pandas y selenium (los otros paquetes no necesitan ser instalados porque ya se encuentran en Python)
# !pip install pandas selenium

# Paquetes para cronometrar el tiempo de ejecución y otros
from timeit import default_timer as timer # Esto devuelve el valor del temporizador de mayor resolución disponible
import datetime
from datetime import timedelta
from itertools import islice # Esto permitirá realizar un "slice" indexado de los diccionarios como si fuesen listas
# Paquetes para el programa (ordenadas ALFABÉTICAMENTE)
from selenium import webdriver
from selenium.webdriver.common.by import By #Método que permite identificar elementos By ("Por")
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import pandas as pd


# Iniciar el cronómetro (me gusta conocer en detalle cuánto tiempo demoró mi programa y el tiempo de extracción promedio)
inicio = timer()

# Opciones de navegación: la siguiente línea inicializa un objeto que será personalizado para las necesidades del programa
options = webdriver.EdgeOptions()
# Este comando maximiza la ventana de navegación y evita problemas del tipo "no se encontró el elemento deseado"
# que se producen cuando uno de los elementos está fuera de la vista en las páginas web y se debe deslizar hacia abajo.
options.add_argument('--start-maximized')
# Este comando es una medida preventiva de "higiene": 
# evita que las extensiones (que podrían variar entre diferentes computadoras) interfieran en la ejecución del programa.
options.add_argument('--disable-extensions')

# Este es una de las partes más importantes
# 1. La parte webdriver.Edge(...) genera un operador web de Edge
# 2. La parte de service=EdgeService(EdgeChromiumDriverManager().install()) se asegura de que el operador esté actualizado
# 2.1. El nombre EdgeChromiumDriverManager() puede resultar confuso, pero simplemente Edge está construido sobre Chromium
# 3. La parte de options=options busca justamente asignar las opciones que habíamos establecido en pasos previos.
driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()), options=options)

# Creamos la lista de nuestros códigos en función del archivo de Excel: 
# Primero importaremos el archivo de excel
CUIdf = pd.read_excel('D:\PYTHON\EjemploInput_Obras_PNISC.xlsx', sheet_name = 0, header = None)
# Segundo convertiremos el objeto en lista
CUI_lista = CUIdf[0].astype(str).values.tolist()

# Diccionario para almacenar los campos de interés y los identificadores que utilizará el programa
# Imagina que el lado izquierdo es como el nombre de la persona y el lado derecho es como su número de DNI
# Formato → CAMPO DE INTERÉS: IDENTIFICADOR WEB
data = {
    # Campos e identificadores en la pestaña de Datos generales
    'CUI'                                                : CUI_lista,       #0 (coloco índices para ayudarme en el slicing)
    'SNIP'                                               : 'td_cu',         #1 
    'NOMBRE DE LA INVERSIÓN'                             : 'td_nominv' ,    #2
    'ESTADO DE LA INVERSIÓN'                             : 'td_estcu',      #3
    'TIPO DE INVERSIÓN'                                  : 'td_tipinv',     #4
    'UNIDAD EJECUTORA DE INVERSIONES (UEI)'              : 'td_uei',        #5
    'COSTO DE INVERSIÓN VIABLE / APROBADO'               : 'td_mtoviab',    #6
    'COSTO DE INVERSIÓN ACTUALIZADO'                     : 'val_cta',       #7
    'COSTO TOTAL DE LA INVERSIÓN ACTUALIZADO'            : 'td_mtototal',   #8
    '¿TIENE EXPEDIENTE TÉCNICO O DOCUMENTO EQUIVALENTE?' : 'td_indet',      #9
    # Campos e identificadores en la pestaña de Ejecución financiera
    'AVANCE FINANCIERO ACUMULADO'                        : 'por_avanacum',  #10
    'DEVENGADO ACUMULADO AL 2024'                        : 'val_efin',      #11
    '¿SE ENCUENTRA PROGRAMADO EN EL PMI?'                : 'td_indpmi',     #12
    'PIM 2024 EN AVANCE FINANCIERO'                      : 'val_pim',       #13
    # Histórico del devengado de la inversión (obtenido a partir de la tabla que se genera en Ejecución financiera)
    'PIM 2024'                                           : '2024',      #14
    'PIM 2023'                                           : '2023',      #15
    'PIM 2022'                                           : '2022'       #16
}

# Ahora creamos otro diccionario con listas para almacenar los campos de interés y sus valores respectivos
# Este primer diccionario añade a la primera columna nuestra lista de CUI, 
# pues es nuestra materia prima, es nuestro único dato conocido
data_lists = {'CUI': CUI_lista}
# Actualizamos el diccionario que generará listas/columnas adicionales 
# para cada Campo del diccionario "data" (el de arriba). 
# Sin embargo, utilizamos islice() para no afectar el elemento 0 de nuestro diccionario que sería {'CUI': CUI_lista}
data_lists.update({key: [] for key in islice(data.keys(), 1, None)})

# Colocamos el tiempo de espera que toleraríamos para que cada CUI cargue en el SSI
espera = 30

# Inicializamos el navegador con el enlace del SSI
driver.get('https://ofi5.mef.gob.pe/ssi/ssi/Index')

# Este bucle itera sobre cada uno de los CUI a partir de la lista que hemos creado desde nuestro archivo Excel
for CUI in CUI_lista:
    
    # Esta es una parte importantísima del código. De buenas a primeras, asumiremos que cada código que buscamos nos
    # genera errores. Luego, el programa irá sustituyendo todos los valores a medida que vaya encontrando la información
    # La última vez que realicé el procedimiento, solo acontecieron errores en 2 de los 1349 códigos ~0.15%
    # Cuando el programa termina de correr, el usuario puede verificar qué códigos faltan y completar manualmente 
    # su base de datos
    for key, value in islice(data.items(), 1, None):
        data_lists[key].append('Ocurrió un error')

    # La dupla "try - except" le proporciona resiliencia al código: PRIMERO intentará realizar el procedimiento usual
    # que consiste en 
    # (1) limpiar el recuadro, 
    # (2) escribir el CUI, 
    # (3) dar click al botón de búsqueda y 
    # (4) esperar qu cargue
        
    try:
        # Cabe mencionar que, los strings que se encuentran hacia el final de cada uno de estos comandos,
        # son como el número de DNI que sirven para identificar a los elementos de la página web:
        # por ejemplo, botones, cuadros de búsqueda, números, tablas, entre otros. 
        
        # Limpiar el recuadro de búsqueda → Necesario porque no es posible escribir sin antes haber limpiado el espacio
        WebDriverWait(driver, espera).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'input#txt_cu'))).clear()
        # Escribir el CUI mediante el comando send_keys()
        WebDriverWait(driver, espera).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input#txt_cu'))).send_keys(CUI)
        # Dar click al botón de búsqueda mediante el método click()
        WebDriverWait(driver, espera).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'i.fa.fa-search'))).click()
        # IMPORTANTÍSIMO: Esperar que termine de cargar; es decir, que el ícono de "cargando" desaparezca
        ## Cuando diseñé por primera vez el programa y aún no había incluido este comando, 
        ## el programa sufría de algunas cargas lentas y obtenía errores (frustrantes) debido a ello 
        WebDriverWait(driver, espera).until(EC.invisibility_of_element_located((By.ID, 'divPreload')))
        # Se utiliza el método islice para iterar sobre nuestro diccionario
        # Sin embargo, será necesario solo considerar los elementos en el rango 1 a 9 que se refieren a elementos dentro
        # de la pestaña "datos generales". Omitir este paso conduciría a errores
        for key, value in islice(data.items(), 1, 10):
            # Identifico el valor del campo de interés gracias al identificador "id" (hay otras formas de identificar)
            # Pero esta es la más fácil
            elemento_texto = driver.find_element("id", value).text
            # Almacena el valor del campo de interés en la lista gracias al método append()
            data_lists[key][-1] = elemento_texto
    
    # Sin embargo, pueden suceder algunos imprevistos como por ejemplo que el proyecto no se encuentre registrado
    # o que el proyecto tenga una demora excesiva al momento de carga (e.g. superior a 1 minuto)
    # En este caso, SEGUNDO, lo que hará el programa es 
    # (1) Asigna valor de 0 a los campos del proyecto problemático
    # (2) Refresca la página para el siguiente proyecto
    except Exception:
        # Recarga la página, pues ante la presencia de algún error, será necesario reiniciar/recargar el sitio web
        driver.refresh()
        continue
    
    # Hacer clic en el botón que nos conduce a la pestaña de Datos Generales
    ## Si bien ya nos encontramos en esta pestaña, es necesario dar este click en apariencia "redundante"
    ## para evitar errores durante la carga porque la página web del SSI no funciona a la perfección y a veces se traba
    ## cuando se omite este paso (lo que conduce a errores)
    xpath_botón_datos_generales = '/html/body/form/div/div[1]/div[1]/div/nav/ul/li[1]/span/img'
    WebDriverWait(driver, espera).until(EC.element_to_be_clickable((By.XPATH,xpath_botón_datos_generales))).click()
    # Hacer clic en el botón que nos conduce a la pestaña de Ejecución Financiera
    xpath_botón_ejecución_financiera = '/html/body/form/div/div[1]/div[1]/div/nav/ul/li[2]/span/img'
    WebDriverWait(driver, espera).until(EC.element_to_be_clickable((By.XPATH, xpath_botón_ejecución_financiera))).click()
    # Esperar que termine de cargar
    WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, 'divPreload')))
    
    # Bucle para campos de interés en la pestaña de Ejecución Financiera - Ibidem con respecto al bucle de Pestaña General
    for key, value in islice(data.items(), 10, 14):
        elemento_texto = driver.find_element("id", value).text # Ibidem con respecto al bucle de Pestaña General
        data_lists[key][-1] = elemento_texto # Ibidem con respecto al bucle de Pestaña General
    
    # Importar la tabla
    try:
        # Verificar que exista la tabla, apoyándose de la existencia de la primera celda ("A1")
        xpath_primera_celda = '/html/body/form/div/div[1]/div[3]/div[2]/div[2]/div/table[2]/tbody/tr[1]/td[1]'
        x = driver.find_element("xpath", xpath_primera_celda)
        # Esperar que terminen de cargar las filas de toda la tabla
        # Es una capa adicional de seguridad en caso de que la página demore o la renderización de la tabla sea lenta
        # No estoy seguro de que sea demasiado necesario, pero por precaución no lo movería
        xpath_filas = "/html/body/form/div/div[1]/div[3]/div[2]/div[2]/div/table[2]/tbody/tr"
        WebDriverWait(driver, espera).until(EC.presence_of_all_elements_located((By.XPATH, xpath_filas)))
        # Obtener las filas de la tabla y hacer un recuento (row count = recuento de filas en inglés) vía len(...) 
        rc = len(driver.find_elements("xpath", xpath_filas))
    
        # Esta sección es importante, pues establece que el valor por defecto de cada uno de los siguientes campos será
        # que no hay información disponible; luego, modificaremos estos valores en caso de que sí haya información.
        # Esto se debe a que en muchas ocasiones las tablas no cuentan con datos sobre los últimos años
        # E.g. la tabla cubre el rango de años 2018-2021 y nuestros años de interés no existen
        data_lists['PIM 2022'][-1] = 'No info. disponible'
        data_lists['PIM 2023'][-1] = 'No info. disponible'
        data_lists['PIM 2024'][-1] = 'No info. disponible'
        
        # A continuación, realizamos un recorrido por cada una de todas las filas de la tabla
        # He econtrado que este método es el más efectivo para lidiar con los diferentes casos que pudiesen surgir
        # Pues en ocasiones la ventana de años disponible varía demasiado
        # Así, en este bucle, cada fila representa un año (columna 1) sus montos asociados (resto de columnas)
        for i in range(1, rc+1):
            # Asignamos el xpath que identifica a la columna 1 (donde se encuentra el año)
            xpath_col_1 = "/html/body/form/div/div[1]/div[3]/div[2]/div[2]/div/table[2]/tbody/tr[" + str(i) + "]/td[1]"
            # Asignamos el xpath que identifica a la columna 3 (donde se encuentra el PIM de dicho año)
            xpath_col_3 = "/html/body/form/div/div[1]/div[3]/div[2]/div[2]/div/table[2]/tbody/tr[" + str(i) + "]/td[3]"
            # Extraemos los valores a partir de la dirección/ID que establecimos en las dos líneas de arriba
            # Recordar, el ".text" busca extraer el texto del elemento
            año = driver.find_element("xpath", xpath_col_1).text
            PIM = driver.find_element("xpath", xpath_col_3).text
            
            # Este multi-condicional tiene como tarea verificar si la fila que estamos navegando corresponde a uno de
            # de nuestros años de interés. En caso afirmativo, procederá a almacenar el valor en campo correspondiente
            if int(año)==2022:
                data_lists['PIM 2022'][-1] = PIM
            elif int(año)==2023:
                data_lists['PIM 2023'][-1] = PIM
            elif int(año)==2024:
                data_lists['PIM 2024'][-1] = PIM
            else:
                continue
    
    # En caso de que la tabla no exista... No hay ninguna información disponible para cualquier año
    # Por ello, rellenamos todos los campos con el mensaje "No info. disponible"
    except NoSuchElementException:
        data_lists['PIM 2022'][-1] = 'No info. disponible'
        data_lists['PIM 2023'][-1] = 'No info. disponible'
        data_lists['PIM 2024'][-1] = 'No info. disponible'
        
# Cerramos el navegador
driver.close()
        
# Convertimos el diccionario con listas en un data frame más manejable
df = pd.DataFrame(data_lists)
# Imprimimos las dimensiones del data frame (esta es una cuestión de preferencias, pero a mí me gusta)
print(f"[{df.shape[0]} rows x {df.shape[1]} columns]")
# Exportamos data frame a un archivo CSV con un nombre compuesto por la fecha y hora (nuevamente, una tema de gustos)
df.to_csv('DB_SSI_' + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S.csv'), index=False)

# Finalizamos el cronómetro
final = timer()
# Imprimir el tiempo de ejecución en la pantalla
print('Duracion total: ' + str(timedelta(seconds = round(final - inicio))))
# Imprimimos el tiempo promedio que demoró el programa en extraer la información de cada obra (o CUI)
print('Tiempo promedio de extracción por CUI: ' + str(round((final - inicio)/len(data_lists['CUI']),2)) + ' segundos')

# Output:
# [1349 rows x 17 columns]
# Duracion total: 0:40:18
# Tiempo promedio de extracción por CUI: 1.79 segundos

[203 rows x 17 columns]
Duracion total: 0:06:02
Tiempo promedio de extracción por CUI: 1.78 segundos
